In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import pickle
import tensorflow as tf

from keras.layers import Input, Embedding, LSTM, Dense, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from IPython.display import SVG
from keras.utils.visualize_util import plot
from keras.utils.visualize_util import model_to_dot

from keras_tqdm import TQDMNotebookCallback

from sklearn import metrics

ImportError: No module named 'matplotlib'

In [ ]:
training_frame = pickle.load( open( '../../Data/... fill in the detailsl', 'rb' ) )

In [ ]:
training_list_1 = list(training_frame.ix[:,1])
training_list_1 = np.array(training_list_1)

training_list_1 = list(training_frame.ix[:,2])
training_list_1 = np.array(training_list_1)

training_list_1 = list(training_frame.ix[:,3])
training_list_1 = np.array(training_list_1)

training_y = list(training_frame.ix[:,4])
training_y = np.array(training_y)

In [ ]:
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'


image_input_1 = Input(shape=(64,64,3), name='image_input_1')
image_input_2 = Input(shape=(64,64,3), name='image_input_2')
image_input_3 = Input(shape=(64,64,3), name='image_input_3')
#image_input_4 = Input(shape=(256,256,3), name='image_input_4')
#image_input_5 = Input(shape=(256,256,3), name='image_input_5')

# shared vision.

generic_input = Input(shape=(64,64,3))

# vision model 1

model = Convolution2D(32, 3, 3, border_mode='same', activation='relu')(generic_input)
model = Convolution2D(32, 3, 3, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

model = Convolution2D(64, 3, 3, border_mode='same', activation='relu')(model)
model = Convolution2D(64, 3, 3, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

model = Convolution2D(128, 3, 3, border_mode='same', activation='relu')(model)
model = Convolution2D(128, 3, 3, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

model = Convolution2D(256, 3, 3, border_mode='same', activation='relu')(model)
model = Convolution2D(256, 3, 3, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

out = Flatten()(model)

vision_model_1 = Model(generic_input, out)

# vision model 2

model = Convolution2D(128, 5, 5, border_mode='same', activation='relu')(generic_input)
model = Convolution2D(128, 5, 5, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

model = Convolution2D(64, 5, 5, border_mode='same', activation='relu')(model)
model = Convolution2D(64, 5, 5, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

model = Convolution2D(32, 3, 3, border_mode='same', activation='relu')(model)
model = Convolution2D(32, 3, 3, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

model = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(model)
model = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

out = Flatten()(model)

vision_model_2 = Model(generic_input, out)


# vision model 3

model = Convolution2D(32, 2, 2, border_mode='same', activation='relu')(generic_input)
model = Convolution2D(32, 2, 2, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

model = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(model)
model = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

model = Convolution2D(8, 3, 3, border_mode='same', activation='relu')(model)
model = Convolution2D(8, 3, 3, border_mode='same', activation='relu')(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

# model = Convolution2D(256, 3, 3, border_mode='same', activation='relu')(model)
# model = Convolution2D(256, 3, 3, border_mode='same', activation='relu')(model)
# model = MaxPooling2D(pool_size=(2, 2))(model)

out = Flatten()(model)

vision_model_3 = Model(generic_input, out)

out_1_1 = vision_model_1(image_input_1)
out_1_2 = vision_model_1(image_input_2)
out_1_3 = vision_model_1(image_input_3)
#out_4 = vision_model(image_input_4)
#out_5 = vision_model(image_input_5)

out_2_1 = vision_model_2(image_input_1)
out_2_2 = vision_model_2(image_input_2)
out_2_3 = vision_model_2(image_input_3)


out_3_1 = vision_model_3(image_input_1)
out_3_2 = vision_model_3(image_input_2)
out_3_3 = vision_model_3(image_input_3)


concatenated_1 = merge([out_1_1, out_1_2, out_1_3], mode='concat')
concatenated_2 = merge([out_2_1, out_2_2, out_2_3], mode='concat')
concatenated_3 = merge([out_3_1, out_3_2, out_3_3], mode='concat')

concatenated = merge([concatenated_1, concatenated_2], mode='concat')

model = Dense(256, activation='relu')(concatenated)
model = Dropout(0.5)(model)

model = Dense(256, activation='relu')(model)
model = Dropout(0.5)(model)

output = Dense(3)(model)
output = Activation('softmax')(output)

classification_model = Model([image_input_1, image_input_2, image_input_3], output)
classification_model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
SVG(model_to_dot(classification_model).create(prog='dot', format='svg'))

In [ ]:
nb_epoch = 10
batch_size = 32

## Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')
history = LossHistory()

In [ ]:
classification_model.fit([training_list_1,training_list_2,training_list_3], training_y, batch_size=batch_size, nb_epoch=nb_epoch,
              validation_split=0.65, verbose=0, shuffle=True, callbacks=[TQDMNotebookCallback(leave_inner = True, leave_outer = True), history])

In [ ]:
loss = history.losses
val_loss = history.val_losses

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('VGG-16 Loss Trend')
plt.plot(loss, 'blue', label='Training Loss')
plt.plot(val_loss, 'green', label='Validation Loss')
plt.xticks(range(0,nb_epoch)[0::2])
plt.legend()
plt.show()

In [ ]:
predictions = classification_model.predict([training_list_1,training_list_2,training_list_3], verbose=0)